In [1]:
# Dependencies
import pandas as pd
from pathlib import Path
import requests
import json
from pprint import pprint
from config import apikey

In [2]:
# Import suburbs' median house prices over the years as a DataFrame
excel_path = Path("Resources/Suburbs.xlsx")
house_prices_df = pd.read_excel(excel_path)
house_prices_df.head()

,Suburbs,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,prelim 2023,change (2021-2022),change(2021-2022),Growth PA
0,ABBOTSFORD,714000,792500,862500,925000,1187500,1280000,1192500,1050000,1200000,1365000,1341500,1409000.0,-2.0,88.0,6.5
1,ABERFELDIE,852500,947500,1045000,1207500,1300000,1471000,1498500,1390000,1520000,1858000,1860000,NaN,0.0,118.0,8.1
2,AINTREE,-,-,-,600000,600000,571000,557500,575000,630000,719000,760000,710000.0,6.0,NaN,NaN
3,AIREYS INLET,634000,664000,625500,680000,715000,737500,869000,985000,1132500,1775000,1725000,1570000.0,-3.0,172.0,10.5
4,AIRPORT WEST,495000,532000,575000,635000,742000,845000,845000,795000,812500,965000,926500,795000.0,-4.0,87.0,6.5


In [3]:
# Lowercase and capitalise suburbs
suburb_list = [suburb.lower().capitalize() for suburb in house_prices_df['Suburbs']]

# Create a new DataFrame with the list of modified suburbs 
suburbs_df = pd.DataFrame({'Suburb': suburb_list})

# Create a new column to put in suburbs' relevant 'place_id' for future search
suburbs_df['Place_id'] = ""
suburbs_df.head()

,Suburb,Place_id
0,Abbotsford,
1,Aberfeldie,
2,Aintree,
3,Aireys inlet,
4,Airport west,


In [4]:
# Set the base URL to search for relevant 'place_id'
base_url = "https://api.geoapify.com/v1/geocode/search"

# Set the API parameters
params = {
    "apiKey":apikey,
    "format":"json",
}

# Loop through each suburb to retrieve its 'place_id'
for index, row in suburbs_df.iterrows():
    # Get the suburb name, and add the State and Country to find location
    suburb_address = row["Suburb"] + ",VIC,Australia"
    # Set a new parameter for the suburb 
    params["text"] = f"{suburb_address}"
    # Make the API request and convert the response to JSON formatting
    response = requests.get(base_url, params=params)
    response = response.json()
    # Modify the DataFrame with the suburbs' relevant 'place_id'
    suburbs_df.loc[index, "Place_id"] = response["results"][0]["place_id"]
    
# Display data to confirm that the 'place_ids' appear
print(suburbs_df.head())

         Suburb                                           Place_id
0    Abbotsford  511d4f159df61f6240593efc4685fbe642c0f00101f901...
1    Aberfeldie  5104c2f3f7b71c62405947b30d373be142c0f00101f901...
2       Aintree  51c04b5fbe6b15624059133b641415dc42c0f00101f901...
3  Aireys inlet  5161a5828a6a03624059056ad6be253b43c0f00101f901...
4  Airport west  5195e6a095451c624059cba4e1ef72dc42c0f00101f901...


In [5]:
# Create new columns to store the number of each type of facility for each suburb
suburbs_df["commercial.shopping_mall"] = ""
suburbs_df["catering"] = ""
suburbs_df["education"] = ""
suburbs_df["entertainment"] = ""
suburbs_df["healthcare"] = ""
suburbs_df["public_transport"] = ""
suburbs_df["office"] = ""
suburbs_df["natural"] = ""

# Create a new list of the new columns 
facilities_list = [facility for facility in suburbs_df.columns]
facilities_list.pop(0)
facilities_list.pop(0)
facilities_list

['commercial.shopping_mall',
 'catering',
 'education',
 'entertainment',
 'healthcare',
 'public_transport',
 'office',
 'natural']

In [6]:
# Set the base url to search for facilities by place_id
base_url_search = "https://api.geoapify.com/v2/places"

# Set key parameter and add to the dictionary in loop
params2 = {"apiKey": apikey}

# Loop through facilities columns to find number of facilities for each suburb
for facility in facilities_list:
    # Set the category as the current facility
    params2["categories"] = facility
    
    # Loop through each suburb 
    for index, row in suburbs_df.iterrows():
        # Set place_id to search as the one for the current suburb
        place_id = row["Place_id"]
        params2["filter"] = f"place:{place_id}"
        # Make the API request and convert the response to JSON formatting
        response2 = requests.get(base_url_search, params=params2).json()
        # Modify the DataFrame with the number of each type of facility
        suburbs_df.loc[index, facility] = len(response2["features"])

suburbs_df.head()

,Suburb,Place_id,commercial.shopping_mall,catering,education,entertainment,healthcare,public_transport,office,natural
0,Abbotsford,511d4f159df61f6240593efc4685fbe642c0f00101f901...,1,20,4,6,4,20,20,6
1,Aberfeldie,5104c2f3f7b71c62405947b30d373be142c0f00101f901...,0,6,4,1,0,0,0,2
2,Aintree,51c04b5fbe6b15624059133b641415dc42c0f00101f901...,0,1,2,0,0,19,0,10
3,Aireys inlet,5161a5828a6a03624059056ad6be253b43c0f00101f901...,0,6,1,0,1,4,0,20
4,Airport west,5195e6a095451c624059cba4e1ef72dc42c0f00101f901...,1,8,3,1,4,20,0,1


In [7]:
# Calculate the total number of facilities for each suburb
suburbs_df["total"] = suburbs_df["commercial.shopping_mall"] + suburbs_df["catering"] + suburbs_df["education"] + suburbs_df["entertainment"] + suburbs_df["healthcare"] + suburbs_df["public_transport"] + suburbs_df["office"] + suburbs_df["natural"]
suburbs_df.head()

,Suburb,Place_id,commercial.shopping_mall,catering,education,entertainment,healthcare,public_transport,office,natural,total
0,Abbotsford,511d4f159df61f6240593efc4685fbe642c0f00101f901...,1,20,4,6,4,20,20,6,81
1,Aberfeldie,5104c2f3f7b71c62405947b30d373be142c0f00101f901...,0,6,4,1,0,0,0,2,13
2,Aintree,51c04b5fbe6b15624059133b641415dc42c0f00101f901...,0,1,2,0,0,19,0,10,32
3,Aireys inlet,5161a5828a6a03624059056ad6be253b43c0f00101f901...,0,6,1,0,1,4,0,20,32
4,Airport west,5195e6a095451c624059cba4e1ef72dc42c0f00101f901...,1,8,3,1,4,20,0,1,38


In [8]:
# Save the data to a CSV file without the index
suburbs_df.to_csv("Resources/suburbs_facilities.csv", index=False)